In [1]:
import cv2
import cv2
import numpy as np
import math
import typing
import matplotlib.pyplot as plt
import os
import pandas as pd
from tqdm import tqdm
def show(frame):
    cv2.imshow("test", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def plot_point(frame,x,y,color=(0,255,0),radius = 0):
    thickness = -1
    return cv2.circle(frame, (x,y), radius, color, thickness)
def plot_line(frame,p1,p2,color=(0,191,255)):
    thickness = 2
    return cv2.line(frame, (p1[0],p1[1]), (p2[0],p2[1]), color, thickness)

# Gets all the contours for certain image

def loadvideo(filename: str) -> np.ndarray:
    """Loads a video from a file.

    Args:
        filename (str): filename of video

    Returns:
        A np.ndarray with dimensions (channels=3, frames, height, width). The
        values will be uint8's ranging from 0 to 255.

    Raises:
        FileNotFoundError: Could not find `filename`
        ValueError: An error occurred while reading the video
    """

    if not os.path.exists(filename):
        raise FileNotFoundError(filename)
    capture = cv2.VideoCapture(filename)

    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    v = np.zeros((frame_count, frame_height, frame_width, 3), np.uint8)

    for count in range(frame_count):
        ret, frame = capture.read()
        if not ret:
            raise ValueError("Failed to load frame #{} of {}.".format(count, filename))

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        v[count] = frame

    v = v.transpose((3, 0, 1, 2))

    return v
def savevideo(filename: str, array: np.ndarray, fps: typing.Union[float, int] = 1):
    """Saves a video to a file.

    Args:
        filename (str): filename of video
        array (np.ndarray): video of uint8's with shape (channels=3, frames, height, width)
        fps (float or int): frames per second

    Returns:
        None
    """

    c, f, height, width = array.shape

    if c != 3:
        raise ValueError("savevideo expects array of shape (channels=3, frames, height, width), got shape ({})".format(", ".join(map(str, array.shape))))
    fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
    out = cv2.VideoWriter(filename, fourcc, fps, (width, height))

    for i in range(f):
        out.write(array[:, i, :, :].transpose((1, 2, 0)))

        
def obtainContourPoints(img):
  # read image

  rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  # set lower and upper bounds on blue color
  lower = (200,0,0)
  upper = (255,200,200)

  # threshold and invert so hexagon is white on black background
  thresh = cv2.inRange(rgb, lower, upper)
  # print(thresh.shape,np.min(thresh),np.max(thresh),type(thresh))
  # show(thresh)
  # get contours
  result = np.zeros_like(thresh)
  im2, contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  #print(im2,contours)
  #cv2.drawContours(img, im2, -1, (0,255,0), 3)

  im2 = np.array(im2)
  index_1 = 0
  index_2 = 0
  for i in im2:
    if i.shape[0]>im2[index_1].shape[0]:
      index_1=index_2
    index_2+=1
  rec = cv2.minAreaRect(im2[index_1])
  color = (0, 0, 255) 
  thickness = 2
  box = cv2.boxPoints(rec)
  box = np.int0(box)
  indexes = [0,1]
  for i in range(0,len(box)):
    for k in range(0,len(indexes)):
      if box[i][1]>box[indexes[k]][1] and not i in indexes:
        indexes[k]=i

  #cv2.drawContours(img,[box],0,(191,0,255),2)
  #img = plot_line(img,box[indexes[0]],box[indexes[1]])
  #img = plot_point(img,box[indexes[0],0],box[indexes[0],1])
  #img = plot_point(img,box[indexes[1],0],box[indexes[1],1])
  #cv2.rectangle(img,rec,color,thickness)
  return img, [box[indexes[0]].tolist(),box[indexes[1]].tolist()]

def calc_ratio(array,vid=None,save = False):
    if save:
        plt.plot(array)
    x = scipy.signal.find_peaks(-array,distance=32)[0]
    ratios = []
    for i in range(0,len(x)):
        if save:
            plt.scatter(x[i],array[x[i]],color='orange')
        if i==len(x)-1:
            y = np.argmax(array[x[i]:])
        else:
            y = np.argmax(array[x[i]:x[i+1]])
        if save:
            plt.scatter(y+x[i],array[y+x[i]],color='red')
        x_val = array[x[i]]
        y_val = array[y+x[i]]
        ratios.append(x_val/y_val)
    if save:
        plt.savefig(os.path.join('Ratio_Calc',vid[:-4]+'.png'))
    return ratios

In [2]:
def obtainThreshPoints(img,iterations = 1):
  # read image

  rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  # set lower and upper bounds on blue color
  lower = (250,0,0)
  upper = (255,200,200)

  # threshold and invert so hexagon is white on black background
  thresh = cv2.inRange(rgb, lower, upper)
  
  #print(np.min(thresh),np.max(thresh),thresh.shape)

  # show(thresh)
  # get contours
  result = np.zeros_like(thresh)
  im2, contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  thresh = cv2.dilate(thresh, None, iterations=iterations)
  #thresh = np.array([thresh,thresh,thresh])
  #thresh = np.transpose(thresh,(1,2,0))
  spare = thresh
  thresh = thresh==255

  img[thresh]=[255,255,255]

  #cv2.drawContours(img, im2, -1, (0,255,0), 3)

  im2 = np.array(im2)
  index_1 = 0
  index_2 = 0
  for i in im2:
    if i.shape[0]>im2[index_1].shape[0]:
      index_1=index_2
    index_2+=1
  rec = cv2.minAreaRect(im2[index_1])
  color = (0, 0, 255) 
  thickness = 2
  box = cv2.boxPoints(rec)
  box = np.int0(box)
  indexes = [0,1]
  for i in range(0,len(box)):
    for k in range(0,len(indexes)):
      if box[i][1]>box[indexes[k]][1] and not i in indexes:
        indexes[k]=i

  # cv2.drawContours(img,[box],0,(191,0,255),2)
  #thresh = plot_line(img,box[indexes[0]],box[indexes[1]])
  #print(np.min(thresh),np.max(thresh),thresh.shape,type(thresh))
  #print(np.min(img),np.max(img),img.shape,type(img))
  #print(iterations,box[indexes[0],1],box[indexes[0],0],spare[box[indexes[0],1],box[indexes[0],0]],box[indexes[1],1],box[indexes[1],0],spare[box[indexes[1],1],box[indexes[1],0]])
  img = plot_point(img,box[indexes[0],0],box[indexes[0],1])
  img = plot_point(img,box[indexes[1],0],box[indexes[1],1])
  #cv2.rectangle(img,rec,color,thickness)
  #show(img)
  x1 = min(box[indexes[0],1],111)
  y1 = min(box[indexes[0],0],111)
  x2 = min(box[indexes[1],1],111)
  y2 = min(box[indexes[1],0],111)
  pair = (not thresh[x1,y1]) or (not thresh[x2,y2])
  return img,thresh, x1,y1,x2,y2, pair

In [3]:
def no_dilation(img,iterations = 1):
  # read image

  rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  # set lower and upper bounds on blue color
  lower = (250,0,0)
  upper = (255,200,200)

  # threshold and invert so hexagon is white on black background
  thresh = cv2.inRange(rgb, lower, upper)
  spare = thresh
  thresh = cv2.dilate(thresh, None, iterations=iterations)
  thresh = cv2.erode(thresh, None, iterations=iterations)
  thresh = thresh==255
  
  return thresh

In [4]:
def midpoint(point1,point2):
    return (point1+point2)/2
def change(points):
    point_arr = [points[0]]
    for i in points[1:]:
        point_arr.append(midpoint(point_arr[-1],i))
    return np.array(point_arr)

In [5]:
def get_points(vid):
    first = np.transpose(vid,(1,2,3,0))
    video = []
    guess = None
    vertexes = []
    for frame in range(0,len(first)):
        #print(temp.Frame==frame,frame)
        img,points = obtainContourPoints(first[frame])
        if np.linalg.norm(points[0])>np.linalg.norm(points[1]):
            c = points[0].copy()
            points[0] = points[1].copy()
            points[1] = c
        vertexes.append(points)

        # video.append(img)
        #show(img)

    #for point in vertexes:
    #change(np.array(vertexes))
    ok = np.array(vertexes)
    # print(ok.shape)
    first_points = change(ok[:,0,:])
    second_points = change(ok[:,1,:])
    return first_points,second_points

In [6]:
def get_dilation(vid,dilations = 5):
    first = np.transpose(vid,(1,2,3,0))
    threshes = []
    vertexes = []
    for frame in first:
        pair = True
        start = 1
        for i in range(0,dilations):#(pair and start<50):
            wow = frame.copy()
            img,thresh,x1,y1,x2,y2,pair = obtainThreshPoints(wow,iterations=start)
            start+=1
        if dilations == 0:
            wow = frame.copy()
            thresh = no_dilation(wow,iterations=1)
        threshes.append(thresh)
    
    return np.array(threshes)

In [7]:
def strain_lengths(vid,threshes,first_points,second_points,filename,output_folder):
    
    first = np.transpose(vid,(1,2,3,0))
    spare = first.copy()
    thresh = threshes
    #print(first.shape,thresh.shape)
    filenames = []
    frame_num = []
    x1s = []
    y1s = []
    error1 = []
    x2s = []
    y2s = []
    error2 = []
    length = []
    angle = []
    
    for frame in range(0,len(first)):
        rgb = cv2.cvtColor(first[frame,:,:], cv2.COLOR_BGR2RGB)
        t = (thresh[frame,:,:]*255).copy()
        t = cv2.inRange(t, 250, 255)
        # print(t.shape,np.min(t),np.max(t),type(t))
        # show(t)
        im2, contours = cv2.findContours(t, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        # print(shortcut,frame,shortcut.index[frame])
        # cv2.drawContours(first[frame], im2, -1, (0,255,0), 3)
        x1 = int(first_points[frame,0])
        y1 = int(first_points[frame,1])
        x2 = int(second_points[frame,0])
        y2 = int(second_points[frame,1])
        def calc_degrees(point):
            return math.atan2(point[1], point[0])
        deg = calc_degrees([x1-x2,y1-y2])
        angle.append(deg)
        final_point = im2[0][0][0]
        index = 0
        count = 0
        for j in im2[0]:
            count+=1
            if np.linalg.norm(j[0]-np.array([x1,y1]))< np.linalg.norm(final_point-np.array([x1,y1])):
                final_point = j[0]
                index = count
        # print(final_point,np.array([x1,y1]), np.linalg.norm(final_point-np.array([x1,y1])),index)
        
        final_point2 = im2[0][0][0]
        index2 = 0
        count2 = 0
        for j in im2[0]:
            count2+=1
            if np.linalg.norm(j[0]-np.array([x2,y2]))< np.linalg.norm(final_point2-np.array([x2,y2])):
                final_point2 = j[0]
                index2 = count2
        # print(final_point2,np.array([x2,y2]), np.linalg.norm(final_point2-np.array([x2,y2])),index2)
        # plot_point(first[frame],x1,y1,color=(255,0,0))
        # plot_point(first[frame],x2,y2,color=(255,0,0))
        # print(im2)
        for k in range(0,index):
            plot_point(first[frame],im2[0][k][0][0],im2[0][k][0][1])
        for k in range(index2,len(im2[0])):
            plot_point(first[frame],im2[0][k][0][0],im2[0][k][0][1])
        for k in range(index,index2):
            plot_point(first[frame],im2[0][k][0][0],im2[0][k][0][1],color=(0,191,255))
        plot_point(first[frame],x1,y1,color=(255,0,255),radius=1)
        plot_point(first[frame],x2,y2,color=(255,0,255),radius=1)
        
        filenames.append(filename)
        frame_num.append(frame)
        x1s.append(final_point[0])
        y1s.append(final_point[1])
        error1.append(np.linalg.norm(final_point-np.array([x1,y1])))
        x2s.append(final_point2[0])
        y2s.append(final_point2[1])
        error2.append(np.linalg.norm(final_point2-np.array([x2,y2])))
        length.append(len(im2[0])-index2+index)
    video = np.transpose(np.array(first),(3,0,1,2))

    savevideo(os.path.join(output_folder,"Strain_Videos",filename),video,fps=30)
    final = pd.DataFrame({"filenames":filenames,'frame_num':frame_num,'x1':x1s,'y1':y1s,'error_1':error1,'x2':x2s,'y2':y2s,'error_2':error2,'length':length,'angle':angle})
    plt.clf()
    plt.plot(frame_num,length)
    plt.title(filename)
    plt.xlabel("Frame")
    plt.ylabel("Length")
    plt.savefig(os.path.join(output_folder,"Strain_Plots",filename[:-4]+'.png'))
    plt.clf()
    return final,calc_ratio(length)

In [8]:
def script(input_folder,output_folder,dilations):
    try:
        os.mkdir(output_folder)
    except:
        print("output folder exists")
    try:
        os.mkdir(os.path.join(output_folder,"Strain_Plots"))
        os.mkdir(os.path.join(output_folder,"Strain_Videos"))
    except:
        print("This folder has been used before")
    output = 'Dilation_5_new'
    iter_num = []
    df = pd.DataFrame()
    Filenames,frame_nums,x1s,y1s,x2s,y2s = [],[],[],[],[],[]
    for vid in tqdm(os.listdir(folder)):
        try:
            video_file = os.path.join(folder,vid)
            first = loadvideo(video_file)[:,:,:,:112]
            left,right = get_points(first)
            thresh = get_dilation(first,dilations = dilations)
            df = df.append(strain_lengths(first,thresh,left,right,vid,output_folder)[0])
        except:
            print(vid)
    df.to_csv(os.path.join(output_folder,"Strain.csv"))

In [9]:
folder = 'J:\\segmented'

In [10]:
for i in range(0,6):
    script(folder,os.path.join("J:\\Strain Dilation Results","Strain_Single_Script_thin_"+str(i)),i)

  0%|                                                                                         | 0/3229 [00:00<?, ?it/s]

output folder exists
This folder has been used before


<ipython-input-1-aec5041280ef>:103: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  im2 = np.array(im2)
  0%|                                                                                 | 1/3229 [00:00<41:54,  1.28it/s]

0X100009310A3BD7FC.avi


  0%|                                                                                 | 2/3229 [00:01<44:11,  1.22it/s]

...

0X3585426448E631FA.avi


 99%|███████████████████████████████████████████████████████████████████████████▍| 3204/3229 [1:06:11<00:30,  1.22s/it]

0X36A70C2755257C5F.avi


100%|████████████████████████████████████████████████████████████████████████████| 3229/3229 [1:06:41<00:00,  1.24s/it]


<Figure size 432x288 with 0 Axes>